### Sample test 1: LBS balance and DOT balance

#### LBS code : https://stats.bis.org/statx/srs/table/A4?c=&p=20191&m=S&f=TSKEYS
<pre>
1. Claim
Q:S:C:A:TO1:A:##:A:5A:A:##:N
    *          |         |
               |         To country
               From country               

2. Liability
Q:S:L:A:TO1:A:##:A:5A:A:##:N
    *          |         |
               |         To country
               From country
               
3. Balance
(Q:S:C:A:TO1:A:##:A:5A:A:##:N) - (Q:S:L:A:TO1:A:##:A:5A:A:##:N)
</pre>


#### DOT code
<pre>
1. Balance
Q:##:B:##
   |    |
   |    To country
   From country
</pre>

### Data retrieval

In [1]:
import pandas as pd
from pymongo import MongoClient

In [2]:
mgclient = MongoClient(unicode_decode_error_handler='ignore')

In [3]:
res = mgclient.lbsdot.countrygroup.find({'code':'OECD'})
oecd_cclist = list(res.next()['cclist'].keys())

In [4]:
# Count number of claims on LBS for OECD countries

num_claim = mgclient.lbsdot.raw_lbsn2.count_documents({
    "FREQ":"Q",
    "L_MEASURE":"S",
    "L_POSITION":"C",  #C: Claim
    "L_INSTR":"A",
    "L_DENOM":"TO1",
    "L_CURR_TYPE":"A",
    "L_PARENT_CTY":{'$in': oecd_cclist},
    "L_REP_BANK_TYPE":"A",
    "L_REP_CTY":"5A",
    "L_CP_SECTOR":"A",
    "L_CP_COUNTRY":{'$in': oecd_cclist},
    "L_POS_TYPE":"N"  #N: Cross-border
})


num_liability = mgclient.lbsdot.raw_lbsn2.count_documents({
    "FREQ":"Q",
    "L_MEASURE":"S",
    "L_POSITION":"L",  #L: Liability
    "L_INSTR":"A",
    "L_DENOM":"TO1",
    "L_CURR_TYPE":"A",
    "L_PARENT_CTY":{'$in': oecd_cclist},
    "L_REP_BANK_TYPE":"A",
    "L_REP_CTY":"5A",
    "L_CP_SECTOR":"A",
    "L_CP_COUNTRY":{'$in': oecd_cclist},
    "L_POS_TYPE":"N"  #N: Cross-border
})

print('#claim:{}, #liability:{}'.format(num_claim, num_liability))

#claim:806, #liability:814


In [5]:
claim_iter = mgclient.lbsdot.raw_lbsn2.find({
    "FREQ":"Q",
    "L_MEASURE":"S",
    "L_POSITION":"C",  #C: Claim
    "L_INSTR":"A",
    "L_DENOM":"TO1",
    "L_CURR_TYPE":"A",
    "L_PARENT_CTY":{'$in': oecd_cclist},
    "L_REP_BANK_TYPE":"A",
    "L_REP_CTY":"5A",
    "L_CP_SECTOR":"A",
    "L_CP_COUNTRY":{'$in': oecd_cclist},
    "L_POS_TYPE":"N"  #N: Cross-border
})

In [6]:
claim_df = pd.DataFrame(list(claim_iter))

In [7]:
claim_df.head(1)

,_id,timeseries,FREQ,L_MEASURE,L_POSITION,L_INSTR,L_DENOM,L_CURR_TYPE,L_PARENT_CTY,L_REP_BANK_TYPE,L_REP_CTY,L_CP_SECTOR,L_CP_COUNTRY,L_POS_TYPE
0,5d3ed151e5c38b7bc2664030,"[[20120630, 5445.263], [20120930, 6194.032], [...",Q,S,C,A,TO1,A,AT,A,5A,A,AT,N


In [55]:
liability_iter = mgclient.lbsdot.raw_lbsn2.find({
    "FREQ":"Q",
    "L_MEASURE":"S",
    "L_POSITION":"L",  #L: Liability
    "L_INSTR":"A",
    "L_DENOM":"TO1",
    "L_CURR_TYPE":"A",
    "L_PARENT_CTY":{'$in': oecd_cclist},
    "L_REP_BANK_TYPE":"A",
    "L_REP_CTY":"5A",
    "L_CP_SECTOR":"A",
    "L_CP_COUNTRY":{'$in': oecd_cclist},
    "L_POS_TYPE":"N"  #N: Cross-border
})

In [57]:
liability_df = pd.DataFrame(list(liability_iter))

In [58]:
liability_df.head(1)

,_id,timeseries,FREQ,L_MEASURE,L_POSITION,L_INSTR,L_DENOM,L_CURR_TYPE,L_PARENT_CTY,L_REP_BANK_TYPE,L_REP_CTY,L_CP_SECTOR,L_CP_COUNTRY,L_POS_TYPE
0,5d3edb1fe5c38b7bc2862ff0,"[[20120630, 9482.634], [20120930, 8974.387], [...",Q,S,L,A,TO1,A,AT,A,5A,A,AT,N


In [79]:
oecd_imfcclist = [x['imfnumeric'] for x in mgclient.lbsdot.countrytable.find({'iso2':{'$in':oecd_cclist}})]
oecd_imfcclist = [str(x) for x in oecd_imfcclist]
imfisocc_map = dict()
for x in mgclient.lbsdot.countrytable.find({'iso2':{'$in':oecd_cclist}}):
    imfisocc_map[str(x['imfnumeric'])] = x['iso2']

In [105]:
dotbalance_iter = mgclient.lbsdot.raw_dot.find({
    'freq': 'Q',
    'Country Code': {'$in': oecd_imfcclist},
    'Indicator Code': 'TBG_USD',
    'Counterpart Country Code': {'$in': oecd_imfcclist}
})

In [106]:
dotbalance_df = pd.DataFrame(list(dotbalance_iter))

In [107]:
dotbalance_df.head(1)

,_id,Country Name,Country Code,Indicator Name,Indicator Code,Counterpart Country Name,Counterpart Country Code,freq,timeseries
0,5d386cee1ce487b9eba5b302,Austria,122,"Goods, Value of Trade Balance, US Dollars",TBG_USD,France,132,Q,"[[19600331, -8300000.0], [19600630, -8500000.0..."


In [48]:
# Save dataframe to pickle files
liability_df.to_pickle('../data/liability_df.pkl')
dotbalance_df.to_pickle('../data/dotbalance_df.pkl')

### Convert to time series

In [9]:
claim_df.head(1)

,_id,timeseries,FREQ,L_MEASURE,L_POSITION,L_INSTR,L_DENOM,L_CURR_TYPE,L_PARENT_CTY,L_REP_BANK_TYPE,L_REP_CTY,L_CP_SECTOR,L_CP_COUNTRY,L_POS_TYPE
0,5d3ed151e5c38b7bc2664030,"[[20120630, 5445.263], [20120930, 6194.032], [...",Q,S,C,A,TO1,A,AT,A,5A,A,AT,N


In [90]:
def convert_timeseries(df, frcol, tocol, data ='lbs'):
    res = list()
    for i in range(df.shape[0]):
        raw = df.iloc[i,:]
        values = [x[1] for x in raw.timeseries]
        if len(values) < 10: # drop if number of values is less 10
            continue
        dates = [x[0] for x in raw.timeseries]
        if raw[frcol] == raw[tocol]:
            continue
        if data == 'dot':
            frto = imfisocc_map[raw[frcol]] + imfisocc_map[raw[tocol]]
        else:
            frto = raw[frcol] + raw[tocol]
        res.append(pd.Series(values, pd.to_datetime(dates, format='%Y%m%d'), name=frto))
        
    return pd.DataFrame(res).transpose()

In [97]:
df_lbs_clam = convert_timeseries(claim_df,'L_PARENT_CTY', 'L_CP_COUNTRY')
df_lbs_liab = convert_timeseries(liability_df,'L_PARENT_CTY', 'L_CP_COUNTRY')

In [76]:
intersect_cols = list(set(df_lbs_clam.columns) & set(df_lbs_liab.columns))
df_lbs_bal = df_lbs_clam[intersect_cols] - df_lbs_liab[intersect_cols]

In [108]:
df_dot_bal = convert_timeseries(dotbalance_df, 'Country Code','Counterpart Country Code', data ='dot')
df_dot_bal = df_dot_bal[intersect_cols]

In [111]:
df_lbs_bal.to_pickle('../data/df_lbs_bal.pkl')
df_dot_bal.to_pickle('../data/df_dot_bal.pkl')

(63, 765)